https://levelup.gitconnected.com/news-classifier-with-naive-bayes-in-python-bbb0c44139e4

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import warnings
warnings.filterwarnings(action='ignore') 

## Naive Bayes
http://scikit-learn.org/stable/modules/naive_bayes.html

### Multinomial Naive Bayes - Scikit-Learn
The Naive Bayes Multinomial classifier is suitable for classification with discrete variables (for example, word counts for text classification). Multinomial distribution typically requires counts of entire entities. However, in practice, fractional counts such as tf-idf can also work.

http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

나이브 베이즈 분류기가 효과적인 이유는 각 특성을 개별로 취급해 파라미터를 학습하고 각 특성에서 클래스별 통계를 단순하게 취합하기 때문입니다. scikit-learn에 구현된 나이브 베이즈 분류기는 GaussianNB, BernoulliNB, MultinomialNB 이렇게 세 가지입니다. 
- GaussianNB는 연속적인 어떤 데이터에도 적용할 수 있고 
- BernoulliNB는 이진 데이터를, 
- MultinomialNB는 카운트 데이터(특성이 어떤 것을 헤아린 정수 카운트로, 예를 들면 문장에 나타난 단어의 횟수입니다)에 적용됩니다. 
                * BernoulliNB, MultinomialNB는 대부분 텍스트 데이터를 분류할 때 사용합니다.

In [8]:
import numpy as np
rng = np.random.RandomState(1)
X = rng.randint(5, size=(6, 100))
y = np.array([1, 2, 3, 4, 5, 6])

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y)

MultinomialNB()

In [10]:
import pandas as pd
pd.DataFrame(X)     # fractional counts such as tf-idf
pd.DataFrame(X[2:3])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,3,4,0,1,3,0,0,1,4,4,...,4,4,1,0,4,2,0,2,4,1
1,1,0,2,4,4,0,4,1,4,1,...,4,3,3,3,3,2,3,3,4,3
2,2,4,4,0,3,3,0,3,1,0,...,2,4,1,2,1,1,1,0,4,4
3,2,2,3,1,4,0,0,3,2,4,...,1,0,2,1,2,0,0,3,4,1
4,0,4,0,3,2,4,3,2,4,2,...,4,2,1,2,3,1,3,3,0,4
5,3,3,3,3,0,2,3,1,3,2,...,4,0,1,0,3,2,1,1,0,0


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,2,4,4,0,3,3,0,3,1,0,...,2,4,1,2,1,1,1,0,4,4


In [9]:
print(clf.predict(X[2:3]))

[3]


### News Classifier
http://qwone.com/~jason/20Newsgroups/

In [11]:
# Imports
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [12]:
# Defining categories 
# (using only 4 out of 20 available so the ranking process is faster)
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [13]:
# Training
twenty_train = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = 42)

In [23]:
# Class
twenty_train.keys()
twenty_train.target_names

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [16]:
len(twenty_train.data)

2257

In [20]:
# Viewing some data (attributes)
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [25]:
# Viewing target variables
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


In [26]:
# Scikit-Learn registers labels as array of numbers in order to increase speed
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

In [27]:
# Viewing the classes of the first 10 records
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


### Bag of Words

In [29]:
twenty_train.data[0]

'From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n'

In [61]:
# Tokenizing
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
count_vect.vocabulary_.get(u'algorithm')  # study (코드 마지막)

X_train_counts.shape
X_train_counts.toarray()

4690

(2257, 35788)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### fit_tranformer

In [56]:
# From occurrences to frequencies - Term Frequency times Inverse Document Frequency (Tfidf)
tf_transformer = TfidfTransformer(use_idf = False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [62]:
# Same result of the previous cell, but combining the functions
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [63]:
# Creating the Multinomial Model
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [64]:
# Predictions
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [65]:
# Creating a Pipeline - Composite Classifier
# vectorizer => transformer => classifier 
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [66]:
# Fit
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

In [67]:
# Model Accuracy
twenty_test = fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True, random_state = 42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646

In [68]:
# Metrics
print(metrics.classification_report(twenty_test.target, predicted, target_names = twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.97      0.60      0.74       319
         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.97      0.81      0.88       396
soc.religion.christian       0.65      0.99      0.78       398

              accuracy                           0.83      1502
             macro avg       0.89      0.82      0.83      1502
          weighted avg       0.88      0.83      0.84      1502



In [69]:
# Confusion Matrix
metrics.confusion_matrix(twenty_test.target, predicted)

array([[192,   2,   6, 119],
       [  2, 347,   4,  36],
       [  2,  11, 322,  61],
       [  2,   2,   1, 393]], dtype=int64)

In [70]:
# Parameters for GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
}

In [71]:
# GridSearchCV
gs_clf = GridSearchCV(text_clf, parameters, n_jobs = -1)

In [72]:
# Fit
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

In [73]:
# Test
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

'soc.religion.christian'

In [74]:
# Score
gs_clf.best_score_

0.9349999999999999

In [76]:
# Parameters used
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.01
tfidf__use_idf: True
vect__ngram_range: (1, 2)


# END

https://scikit-learn.org/stable/modules/feature_extraction.html

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
]
X = vectorizer.fit_transform(corpus)
X
X.toarray()

CountVectorizer()

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

In [48]:
analyze = vectorizer.build_analyzer()
analyze("This is a text document to analyze.") 

['this', 'is', 'text', 'document', 'to', 'analyze']

In [49]:
vectorizer.get_feature_names()

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

In [54]:
pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())

,and,document,first,is,one,second,the,third,this
0,0,1,1,1,0,0,1,0,1
1,0,1,0,1,0,2,1,0,1
2,1,0,0,0,1,0,1,1,0
3,0,1,1,1,0,0,1,0,1


The converse mapping from feature name to column index is stored in the vocabulary_ attribute of the vectorizer:

In [55]:
for i in vectorizer.get_feature_names():
    vectorizer.vocabulary_.get(i)

0

1

2

3

4

5

6

7

8